In [8]:
from qiskit import QuantumCircuit, QuantumRegister, Aer, transpile, assemble
from qiskit.visualization import array_to_latex

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error

from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation
from qiskit.opflow import CircuitSampler
from qiskit import Aer, BasicAer

#General imports
import numpy as np

#Operator Imports
from qiskit.opflow import Z, X, I, StateFn, CircuitStateFn, SummedOp
from qiskit.opflow.gradients import Gradient, NaturalGradient, QFI, Hessian

#Circuit imports
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression
from qiskit.circuit.library import EfficientSU2

from qiskit.opflow import MatrixOp

import scipy.optimize as opt
from qiskit.algorithms.optimizers import SciPyOptimizer

from scipy.optimize import minimize
from itertools import product

In [9]:
def circuit(string = "0000", n_layers = 4, n_qubits = 4):
    params = ParameterVector('theta', length=n_layers*n_qubits)
    it = iter(params)
    C = QuantumCircuit(n_qubits)
    C.initialize(string)

    for j in range(n_layers):
        for i in range(n_qubits):
            C.rx(next(it), i)

        for i in range(n_layers-1):
            C.cx(i, i+1)

        C.barrier()
    return C, params

In [31]:
diag = np.arange(-1, 15)
H = MatrixOp(np.diag(diag)).to_pauli_op()

backend = Aer.get_backend('qasm_simulator')
q_instance = QuantumInstance(backend=backend)#, shots=8192)

n_qubits = 4
n_layers = 2


def loss_fct(value):
    out = 0
    for i, string in enumerate([''.join(p) for p in product('10', repeat=n_qubits)]):
        circ1, _ = circuit(string = string, n_qubits = n_qubits, n_layers = n_layers)
        circ1 = circ1.decompose()
        circ1_bound = circ1.bind_parameters(value)
        psi = CircuitStateFn(primitive=circ1_bound, coeff=1.)
        measurable_expression = StateFn(H, is_measurement=True).compose(psi) 
        expectation = PauliExpectation().convert(measurable_expression)  
        sampler = CircuitSampler(q_instance).convert(expectation) 
        energy = sampler.eval().real
        out += (energy - diag[i])**2
    return out

params_init = 4.
#O = SciPyOptimizer("COBYLA")
#O.optimize(loss_fct, params)
value = np.random.rand(n_qubits*n_layers)
result = minimize(loss_fct, value, method="COBYLA")

In [28]:
values_end = result.x
circ1, _ = circuit(string = "0011", n_qubits = n_qubits, n_layers = n_layers)
circ1_bound = circ1.bind_parameters(values_end)
op = ~StateFn(H) @ CircuitStateFn(primitive=circ1_bound, coeff=1.)
energy = op.eval()
energy

(11.004414194422688+0j)